# Create dataset

The **data** path is a unzipped folder from the exercise zip file

In [10]:
import torch
from datasets import load_dataset
import datasets
from torch.utils.data import DataLoader

dataset = load_dataset("audiofolder", data_dir="data")

Resolving data files:   0%|          | 0/12798 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1000 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1000 [00:00<?, ?it/s]

In [58]:
len(dataset['train'][4]['audio']["array"])

15019

## Upload it to the hub

In [17]:
dataset.push_to_hub("MrObay/Audio-ex3", data_dir="raw")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/12798 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/128 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

D:\Documents\GitHub\Audio-ex3\.venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sean\.cache\huggingface\hub\datasets--MrObay--Audio-ex3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/MrObay/Audio-ex3/commit/ac00ab05ebbcac43af5596bc3981ad4666c34057', commit_message='Upload dataset', commit_description='', oid='ac00ab05ebbcac43af5596bc3981ad4666c34057', pr_url=None, pr_revision=None, pr_num=None)

---
# Create MFCC features

In [1]:
from datasets import load_dataset
dataset = load_dataset("MrObay/Audio-ex3", "raw")

In [13]:
from loader import MFCC_FEATURES, zero_to_nine, AUDIO_LENGTH
import librosa
import numpy as np
from transformers import BertTokenizer

def preprocess_function(example):
    # pad
    audio = example["audio"]["array"]
    audio = np.pad(audio, (0, AUDIO_LENGTH - len(audio)), mode="constant")
    mfcc = librosa.feature.mfcc(audio, sr=22050, n_mfcc=MFCC_FEATURES)
    example["mfcc"] = mfcc
    example["label_str"] = zero_to_nine[example["label"]]
    return batch

mf_dataset = dataset.map(preprocess_function, batched=False)

D:\Documents\GitHub\Audio-ex3\.venv\lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Map:   0%|          | 0/12798 [00:00<?, ? examples/s]


KeyboardInterrupt



## Upload it to the hub

In [1]:
mf_dataset.push_to_hub("MrObay/Audio-ex3")

NameError: name 'mf_dataset' is not defined

## Load the dataset

In [5]:
from datasets import load_dataset
mfcc_dataset = load_dataset("MrObay/Audio-ex3")
mfcc_dataset = mfcc_dataset.remove_columns(["audio", "label"])

## Data loader

In [9]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        
    def __getitem__(self, index):
        x = self.dataset[index]['mfcc']
        y = self.dataset[index]['label']
        return x, y
    
    def __len__(self):
        return len(self.dataset)
    
    
def collate_fn(batch):
    return [torch.stack([x for x,_ in batch]), torch.stack([y for _,y in batch])]

In [15]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
# data = MyDataset(mfcc_dataset['test'])


dataT = TensorDataset(torch.tensor(mfcc_dataset['test']["mfcc"]))
dataLoader = DataLoader(dataT, batch_size=64, shuffle=True, collate_fn=collate_fn)

KeyboardInterrupt: 

In [26]:
import torch
for batch in dataLoader:
    x, y = batch
    print(x.shape)

torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([64, 13, 32])
torch.Size([40, 13, 32])
